# Fallbacks

When working with language models, you may often encounter issues from the underlying APIs, whether these be rate limiting or downtime. Therefor, as you go to move your LLM applications into production it becomes more and more important to safe guard against these. That's why we've introduced the concept of fallbacks.

Crucially, fallbacks can be applied not only on the LLM level but on the whole runnable level. This is important because often times different models require different prompts. So if your call to OpenAI fails, you don't just want to send the same prompt to Anthropic - you probably want want to use a different prompt template and send a different version there.

In [2]:
from langchain.chat_models import ChatOpenAI

bad_llm = ChatOpenAI(model_name="gpt-fake")
good_llm = ChatOpenAI(model_name="gpt-3.5-turbo")
llm = bad_llm.with_fallbacks([good_llm])

result = llm.invoke("Why did the the chicken cross the road?")

In [4]:
result

AIMessage(content='To get to the other side.', additional_kwargs={}, example=False)

We can add multiple fallbacks

In [3]:
llm = bad_llm.with_fallbacks([bad_llm, bad_llm, good_llm])

result = llm.invoke("Why did the the chicken cross the road?")

In [5]:
result

AIMessage(content='To get to the other side.', additional_kwargs={}, example=False)

We can use our "LLM with Fallbacks" as we would a normal LLM.

In [6]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You're a nice assistant who always includes a compliment in your response"),
        ("human", "Why did the {animal} cross the road"),
    ]
)
chain = prompt | llm
chain.invoke({"animal": "kangaroo"})

AIMessage(content="That's a great question! Kangaroos are known for their incredible hopping abilities, so maybe this particular kangaroo wanted to show off its impressive skills while crossing the road. I must say, kangaroos are truly fascinating creatures!", additional_kwargs={}, example=False)

We can also create fallbacks for sequences, that are sequences themselves. Here we do that with two different models: ChatOpenAI and then normal OpenAI (which does not use a chat model). Because OpenAI is NOT a chat model, you likely want a different prompt.

In [ ]:
# First let's create a chain with a ChatModel
# We add in a string output parser here so the outputs between the two are the same type
from langchain.schema.output_parser import StrOutputParser

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You're a nice assistant who always includes a compliment in your response"),
        ("human", "Why did the {animal} cross the road"),
    ]
)
chat_model = ChatOpenAI(model_name="gpt-fake")
bad_chain = chat_prompt | chat_model | StrOutputParser()

In [7]:
# Now lets create a chain with the normal OpenAI model
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

prompt_template = """Instructions: You should always include a compliment in your response.

Question: Why did the {animal} cross the road?"""
prompt = PromptTemplate.from_template(prompt_template)
llm = OpenAI()
good_chain = prompt | llm

In [9]:
# We can now create a final chain which combines the two
chain = bad_chain.with_fallbacks([good_chain])
chain.invoke({"animal": "turtle"})

"\n\nAnswer: He must have been in a hurry to get somewhere, but I'm sure he was careful while crossing. Good for him!"